In [26]:
def iou(a, b):
    """Compute the Intersection-Over-Union of two sets of boxes."""
    area_a = (a[:, 3] - a[:, 1]) * (a[:, 2] - a[:, 0])
    area_b = (b[:, 3] - b[:, 1]) * (b[:, 2] - b[:, 0])

    intersection_top_left = torch.max(a[:, None, :2], b[:, :2])
    intersection_bottom_right = torch.min(a[:, None, 2:], b[:, 2:])
    intersection_area = (intersection_bottom_right - intersection_top_left).clamp(min=0).prod(2)

    return intersection_area / (area_a[:, None] + area_b - intersection_area)

def compute_tp_fp(predictions, ground_truth, iou_threshold=0.5):
    """Compute the number of true positives and false positives."""
    tp = 0
    fp = 0
    gt_flags = [0] * len(ground_truth)  # Flags to mark if a ground truth box is matched

    for pred in predictions:
        max_iou = 0
        max_idx = -1
        for idx, gt in enumerate(ground_truth):
            curr_iou = iou(pred['bbox'].unsqueeze(0), gt['bbox'].unsqueeze(0)).item()
            if curr_iou > max_iou:
                max_iou = curr_iou
                max_idx = idx

        if max_iou >= iou_threshold:
            if gt_flags[max_idx] == 0:  # If this ground truth hasn't been matched before
                tp += 1
                gt_flags[max_idx] = 1
            else:
                fp += 1  # This ground truth has been matched before, so it's a false positive
        else:
            fp += 1

    return tp, fp

def get_precision(predictions, ground_truth, unknown=True):
    # Compute TP and FP for ensemble predictions of the unknown class
    if unknown is not None:
        if unknown:
            ground_truth = [gt for gt in ground_truth if gt['label_id'] == 80]
            predictions = [pred for pred in predictions if pred['label_id'] == 80]
        else: 
            ground_truth = [gt for gt in ground_truth if gt['label_id'] != 80]
            predictions = [pred for pred in predictions if pred['label_id'] != 80]
    tp, fp = compute_tp_fp(predictions, ground_truth)
    
    # Calculate precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    return precision
    

def get_recall(predictions, ground_truth, iou_threshold=0.1, unknown=True):
    """Compute recall for the ensemble predictions."""
    if unknown is not None:
        if unknown:
            ground_truth = [gt for gt in ground_truth if gt['label_id'] == 80]
            predictions = [pred for pred in predictions if pred['label_id'] == 80]
        else: 
            ground_truth = [gt for gt in ground_truth if gt['label_id'] != 80]
            predictions = [pred for pred in predictions if pred['label_id'] != 80]
            
    tp, fp = compute_tp_fp(predictions, ground_truth, iou_threshold)
    fn = len(ground_truth) - tp
    
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    return recall


In [27]:
import os

directory_path = "output/from_dataset/filtered_unknown/SOWODB/"

# List all files in the directory
all_files = os.listdir(directory_path)

# Filter the files based on the desired suffix
files = [f.split("_")[0] for f in all_files if f.endswith("t1.pth.txt")]

print(len(files))


4952


In [28]:
unknown_recall = []
known_precision = []
general_recall = []
general_precision = []

for filename in files:
    filepaths = {
    "ground_truth": f"output/from_dataset/filtered_unknown/SOWODB/{filename}_gt.txt",
    "model_1": f"output/from_dataset/filtered_unknown/SOWODB/{filename}_t1.pth.txt",
    }
    # Storing the data from each file in a dictionary
    data = {}
    for key, filepath in filepaths.items():
        with open(filepath, "r") as file:
            data[key] = file.readlines()
    
    h, w = map(int, data["ground_truth"][-1].strip().split(": ")[1].split(", "))
    
    # Displaying the first few lines of each file to understand the data format
    sample_data = {key: value[:5] for key, value in data.items()}
    import re
    import torch
    
    def parse_bbox(tensor_string):
        """Parse tensor string to extract bounding box coordinates."""
        coords = list(map(float, re.findall(r'([-+]?\d*\.\d+e[+-]?\d+|[-+]?\d*\.\d+|[-+]?\d+)', tensor_string)))
        return torch.tensor(coords)
    
    def parse_data_line(line):
        """Parse a line from the data files to extract bounding box, label, ID, and probability."""
        bbox = parse_bbox(re.search(r'tensor\((.*?)\)', line).group(0))
        label = re.search(r'LABEL: (.*?)(?: \(|$)', line).group(1)
        label_id = int(re.search(r'ID: (\d+)', line).group(1))
        prob = float(re.search(r'PROB: (.+)', line).group(1))
        return {'bbox': bbox, 'label': label, 'label_id': label_id, 'prob': prob}
    
    
    # Parsing the data again
    parsed_data = {key: [parse_data_line(line) for line in value[:-1]] for key, value in data.items()}
    unknown_recall.append(get_recall(parsed_data['model_1'], parsed_data['ground_truth'], unknown=True))
    known_precision.append(get_precision(parsed_data['model_1'], parsed_data['ground_truth'], unknown=False))
    # general_recall.append(get_recall(parsed_data['model_1'], parsed_data['ground_truth'], unknown=None))
    # general_precision.append(get_precision(parsed_data['model_1'], parsed_data['ground_truth'], unknown=None))


In [29]:
def mean(lst): 
    return sum(lst) / len(lst) 
 
# Driver Code 
print(mean(unknown_recall))
print(mean(known_precision))
print(mean(general_recall))
print(mean(general_precision))

0.14416783837650218
0.5043838308074982
0.36729391147968204
0.2782444016626013
